<a href="https://www.kaggle.com/code/meghanshuverma/textanomaly-experiment-mixed?scriptVersionId=211176587" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/binary-sst2-dataset/val.csv
/kaggle/input/binary-sst2-dataset/unsup.csv
/kaggle/input/binary-sst2-dataset/train.csv
/kaggle/input/binary-sst2-dataset/test.csv


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
import json
import bz2
import pickle
import os

In [3]:
data_train = pd.read_csv('/kaggle/input/binary-sst2-dataset/train.csv')
data_test = pd.read_csv('/kaggle/input/binary-sst2-dataset/test.csv')

data_train = data_train[:]
data_test = data_test[:]
print(data_train.head())

   label                                           sentence
0      1  A stirring, funny and finally transporting re-...
1      0  Apparently reassembled from the cutting-room f...
2      0  They presume their audience won't sit still fo...
3      1  This is a visually stunning rumination on love...
4      1  Jonathan Parker's Bartleby should have been th...


In [4]:
# Assuming data_train is a DataFrame and labels are in the 'labels' column
filtered_data = data_train[data_train['label'] == 1]

# Display the filtered data
print(filtered_data)

      label                                           sentence
0         1  A stirring, funny and finally transporting re-...
3         1  This is a visually stunning rumination on love...
4         1  Jonathan Parker's Bartleby should have been th...
5         1  Campanella gets the tone just right -- funny i...
7         1  Béart and Berling are both superb, while Huppe...
...     ...                                                ...
6909      1  Tambor and Clayburgh make an appealing couple ...
6911      1  The best thing about the movie is its personab...
6912      1  Here is a divine monument to a single man's st...
6915      1  Painful, horrifying and oppressively tragic, t...
6919      1  A deliciously nonsensical comedy about a city ...

[3610 rows x 2 columns]


In [5]:
data_train.head(5)

,label,sentence
0,1,"A stirring, funny and finally transporting re-..."
1,0,Apparently reassembled from the cutting-room f...
2,0,They presume their audience won't sit still fo...
3,1,This is a visually stunning rumination on love...
4,1,Jonathan Parker's Bartleby should have been th...


In [6]:
data_train.rename(columns={'sentence':'title','label':'tag'},inplace=True)
data_test.rename(columns={'sentence':'title','label':'tag'},inplace=True)

In [7]:
# def make_tags(x):   #converting the ratings column into 0's and 1's.  for binary classifier to take place
#     if(x=="negative"):
#         return 0
#     else:
#         return 1

In [8]:
# data_train['tag']=data_train['tag'].apply(lambda x: make_tags(x))
# data_test['tag']=data_test['tag'].apply(lambda x: make_tags(x))

In [9]:
count0 = (data_train['tag']==0).sum()
count1 = (data_train['tag']==1).sum()

if(count0 > count1):
    imbalance_ratio = (count0)/count1
else:
    imbalance_ratio = (count1)/count0

print('imbalance_ratio',imbalance_ratio)

imbalance_ratio 1.0906344410876132


In [10]:
y_test_normal = data_test['tag']
y_test_normal

0       0
1       0
2       0
3       0
4       1
       ..
1816    0
1817    0
1818    0
1819    0
1820    0
Name: tag, Length: 1821, dtype: int64

In [11]:
def no_of_words_in_paragraph(x):
    return len(list(x))


data_train['no_of_words_in_paragraph']=data_train['title'].apply(lambda x:no_of_words_in_paragraph(x))
data_test['no_of_words_in_paragraph']=data_test['title'].apply(lambda x:no_of_words_in_paragraph(x))


print(data_train)
avg=data_train['no_of_words_in_paragraph'].mean()
maxim=data_train['no_of_words_in_paragraph'].max()
print('average paragraph length',data_train['no_of_words_in_paragraph'].mean())
print('maximum para length',data_train['no_of_words_in_paragraph'].max())
excess=(data_train['no_of_words_in_paragraph']>avg).sum()
excess_ratio=excess/len(data_train)
print('excess_ratio',excess_ratio)

      tag                                              title  \
0       1  A stirring, funny and finally transporting re-...   
1       0  Apparently reassembled from the cutting-room f...   
2       0  They presume their audience won't sit still fo...   
3       1  This is a visually stunning rumination on love...   
4       1  Jonathan Parker's Bartleby should have been th...   
...   ...                                                ...   
6915    1  Painful, horrifying and oppressively tragic, t...   
6916    0  Take Care is nicely performed by a quintet of ...   
6917    0  The script covers huge, heavy topics in a blan...   
6918    0  A seriously bad film with seriously warped log...   
6919    1  A deliciously nonsensical comedy about a city ...   

      no_of_words_in_paragraph  
0                          102  
1                           77  
2                          221  
3                          101  
4                           97  
...                        ...  


In [12]:
!pip install nltk

In [13]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [14]:
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/PY3/english.pickle')
# Loading PunktSentenceTokenizer using English pickle file
def make_sent_token(x):
    return tokenizer.tokenize(x)

In [15]:
data_train['sentence_token']=data_train['title'].apply(lambda x: make_sent_token(x))
data_test['sentence_token']=data_test['title'].apply(lambda x: make_sent_token(x))

In [16]:
data_train['no_of_sentences']=data_train['sentence_token'].apply(lambda x:len(x))
data_test['no_of_sentences']=data_test['sentence_token'].apply(lambda x:len(x))

In [17]:
avg_sen_length=data_train['no_of_words_in_paragraph'].sum()/data_train['no_of_sentences'].sum()
print(avg_sen_length)

100.00329796386579


In [18]:
def max_length_of_sentence(x,y):
    sen=x
    nu=y
    #print(sen)
    ma=0
    if(nu>1):
        l=sen.split('.')
        #print(l)
        for i in range(len(l)):
            k=l[i].replace(',','')
            maxi=len(k.split())
            #print(maxi)
            if(maxi>ma):
                ma=maxi
        return ma
    else:
        return len(sen.split())

In [19]:
data_train['max_words_in_sentence']=data_train.apply(lambda x: max_length_of_sentence(x.title,x.no_of_sentences),axis=1)
data_test['max_words_in_sentence']=data_test.apply(lambda x: max_length_of_sentence(x.title,x.no_of_sentences),axis=1)

In [20]:
x1=max(data_train['no_of_sentences'])
y1=max(data_train['max_words_in_sentence'])

x2=max(data_test['no_of_sentences'])
y2=max(data_test['max_words_in_sentence'])

if(x1>=x2):
    m=x1
    print(m)
    m=m
else:
    m=x2
    m=m

if(y1>=y2):
    n=y1
else:
    n=y2

#So each para will be converted to a m*n matrix
if(m<5):
    m=6
else:
    m+=2
print('x1,x2,y1,y2',x1,x2,y1,y2)

print("m-->",m,n)

2
x1,x2,y1,y2 2 2 49 52
m--> 6 52


In [21]:
import re
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
def make_tokens(text):
    return word_tokenize(text)
from tqdm import tqdm

# Make sure to enable `tqdm` for pandas
tqdm.pandas()

data_train['tokens']=data_train['title'].progress_apply(lambda x: make_tokens(x))
data_test['tokens']=data_test['title'].progress_apply(lambda x: make_tokens(x))

100%|██████████| 1821/1821 [00:00<00:00, 6410.66it/s]


In [22]:
!git clone https://github.com/DonaldRR/SimpleNet

Cloning into 'SimpleNet'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 95 (delta 42), reused 31 (delta 31), pack-reused 39 (from 1)
Unpacking objects: 100% (95/95), 235.89 KiB | 4.07 MiB/s, done.


In [23]:
!mkdir -p glove6b300dtxt
!kaggle datasets download -d thanakomsn/glove6b300dtxt -p glove6b300dtxt

Dataset URL: https://www.kaggle.com/datasets/thanakomsn/glove6b300dtxt
License(s): DbCL-1.0
 99%|████████████████████████████████████████▍| 381M/386M [00:01<00:00, 280MB/s]
100%|█████████████████████████████████████████| 386M/386M [00:01<00:00, 277MB/s]


In [24]:
!unzip glove6b300dtxt/glove6b300dtxt.zip -d glove6b300dtxt

Archive:  glove6b300dtxt/glove6b300dtxt.zip
  inflating: glove6b300dtxt/glove.6B.300d.txt  


In [25]:
all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))
para_max=max(training_sentence_lengths)

vocab=len(TRAINING_VOCAB)

133321 words total, with a vocabulary size of 16449
Max sentence length is 53


In [26]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), char_level=False)
tokenizer.fit_on_texts(data_train['title'])       # we assigned values

In [27]:
train_word_index = tokenizer.word_index

In [28]:
def make_train_seq(x):
    return tokenizer.texts_to_sequences(x)

data_train['train_seq']=data_train['sentence_token'].progress_apply(lambda x:make_train_seq(x) )
data_test['train_seq']=data_test['sentence_token'].progress_apply(lambda x:make_train_seq(x) )

100%|██████████| 1821/1821 [00:00<00:00, 51857.47it/s]


In [29]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
def padding(x):    #now padding each sentence to a length of n...number of columns
    MAX_SENTENCE_LENGTH=n  #(no of columns)
    return pad_sequences(x,maxlen=MAX_SENTENCE_LENGTH,padding='post')

data_train['padded']=data_train['train_seq'].apply(lambda x:padding(x))
data_test['padded']=data_test['train_seq'].apply(lambda x:padding(x))

In [30]:
embeddings_index = {}
f = open('glove6b300dtxt/glove.6B.300d.txt')
for line in f:
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('GloVe data loaded')

GloVe data loaded


In [31]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.initializers import Constant

## EMBEDDING_DIM =  ## seems to need to match the embeddings_index dimension
EMBEDDING_DIM = embeddings_index.get('a').shape[0]
print(EMBEDDING_DIM)
#num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
 #= np.zeros(len(train_word_index) + 1, EMBEDDING_DIM)
train_embedding_weights = np.zeros((len(train_word_index)+1,
 EMBEDDING_DIM))
for word, i in train_word_index.items():
    #print("sd")
    embedding_vector = embeddings_index.get(word) ## This references the loaded embeddings dictionary
    if embedding_vector is not None:
        train_embedding_weights[i] = embedding_vector
print(train_embedding_weights.shape)

300
(14516, 300)


In [32]:
def make_full_para(x):  
    l=len(x)
    h=m-l  
    z=[0]*h*n   
    z=np.reshape(z,(h,n)) 
    s=x.tolist()+z.tolist()
    return s

In [33]:
from tqdm import tqdm

# Make sure to enable `tqdm` for pandas
tqdm.pandas()
data_train['full_para']=data_train['padded'].progress_apply(lambda x : make_full_para(x))
data_test['full_para']=data_test['padded'].progress_apply(lambda x : make_full_para(x))

100%|██████████| 1821/1821 [00:00<00:00, 31698.28it/s]


In [34]:
def create_1d_para(x):
    l=[]
    for i in x:
        l+=i    #concatenating all the sentences in a para into a single 1 d arrray
    return l

In [35]:
data_train['single_d_array']=data_train['full_para'].progress_apply(lambda x: create_1d_para(x) )
data_test['single_d_array']=data_test['full_para'].progress_apply(lambda x: create_1d_para(x) )

100%|██████████| 1821/1821 [00:00<00:00, 120679.85it/s]


In [36]:
train_cnn_data=np.array(data_train['single_d_array'].tolist())
test_cnn_data=np.array(data_test['single_d_array'].tolist())

In [37]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

# Parameters
filter_sizes = [1,2,3,4]
num_filters = 32
embed_size=300
embedding_matrix=train_embedding_weights
max_features=len(train_word_index)+1
maxlen = m*n  # Replace with `m * n` (provide your value for m and n)

# Model Input
input_layer = layers.Input(shape=(maxlen,), name="Input")

# Embedding Layer with Pre-trained Embeddings
embedding_layer = layers.Embedding(
    input_dim=max_features,
    output_dim=embed_size,
    weights=[embedding_matrix],
    input_length=maxlen,
    trainable=False,
    name="Embedding",
)(input_layer)

# 1. Feature Extractor: CNN-based with dynamic filter sizes
conv_blocks = []
for filter_size in filter_sizes:
    conv = layers.Conv1D(
        filters=num_filters,
        kernel_size=filter_size,
        activation="relu",
        padding="same",
        name=f"Conv1D_Filter{filter_size}",
    )(embedding_layer)
    conv_blocks.append(conv)

# Concatenate the outputs of different filter sizes
x = layers.Concatenate(name="ConcatConv")(conv_blocks)

# Optionally apply max pooling to downsample the sequence length
x = layers.MaxPooling1D(pool_size=2, name="MaxPooling")(x)

# 2. Feature Adaptor: Linear projection to adapted features
adapted_dim = 128
x_adapted = layers.Dense(adapted_dim, activation="relu", name="FeatureAdaptor")(x)

# 3. Anomalous Feature Generator (Training Only)
# Add Gaussian noise to simulate anomalies
noise = layers.GaussianNoise(stddev=0.1, name="GaussianNoise")(x_adapted)

# 4. Discriminator: Classify normal vs anomalous features
# Shared discriminator logic
discriminator_input = layers.Input(shape=(maxlen // 2, adapted_dim), name="DiscriminatorInput")
d = layers.Conv1D(64, kernel_size=3, activation="relu", name="DiscriminatorConv1")(discriminator_input)
d = layers.GlobalMaxPooling1D(name="DiscriminatorPooling")(d)
d = layers.Dense(64, activation="relu", name="DiscriminatorDense1")(d)
d = layers.Dense(1, activation="sigmoid", name="DiscriminatorOutput")(d)
discriminator = models.Model(discriminator_input, d, name="Discriminator")

# Apply discriminator on both normal and noisy features
normal_scores = discriminator(x_adapted)
anomalous_scores = discriminator(noise)

# 5. Anomaly Map Generation
# Outputs anomaly scores for words (local) and the entire sentence (global)
local_anomaly_map = layers.Dense(1, activation="sigmoid", name="LocalAnomalyMap")(x_adapted)
global_anomaly_map = layers.GlobalAveragePooling1D(name="GlobalAnomalyMap")(local_anomaly_map)

# Final Model Outputs
model = models.Model(
    inputs=input_layer,
    outputs=[normal_scores, anomalous_scores, global_anomaly_map],
    name="AnomalyDetectionModel"
)

# Compile the Model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=["binary_crossentropy", "binary_crossentropy", "mse"],
    loss_weights=[0.5, 0.5, 0.1],
)

# Model Summary
model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "AnomalyDetectionModel"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Input (InputLayer)  │ (None, 312)       │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Embedding           │ (None, 312, 300)  │  4,354,800 │ Input[0][0]       │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1D_Filter1      │ (None, 312, 32)   │      9,632 │ Embedding[0][0]   │
│ (Conv1D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1D_Filter2      │ (None, 312, 32)   │     19,232 │ Embedding[0][0]   │
│ (Conv1D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1D_Filter3      │ (None, 312, 32)   │     28,832 │ Embedding[0][0]   │
│ (Conv1D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1D_Filter4      │ (None, 312, 32)   │     38,432 │ Embedding[0][0]   │
│ (Conv1D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ConcatConv          │ (None, 312, 128)  │          0 │ Conv1D_Filter1[0… │
│ (Concatenate)       │                   │            │ Conv1D_Filter2[0… │
│                     │                   │            │ Conv1D_Filter3[0… │
│                     │                   │            │ Conv1D_Filter4[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ MaxPooling          │ (None, 156, 128)  │          0 │ ConcatConv[0][0]  │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ FeatureAdaptor      │ (None, 156, 128)  │     16,512 │ MaxPooling[0][0]  │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ GaussianNoise       │ (None, 156, 128)  │          0 │ FeatureAdaptor[0… │
│ (GaussianNoise)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ LocalAnomalyMap     │ (None, 156, 1)    │        129 │ FeatureAdaptor[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Discriminator       │ (None, 1)         │     28,865 │ FeatureAdaptor[0… │
│ (Functional)        │                   │            │ GaussianNoise[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ GlobalAnomalyMap    │ (None, 1)         │          0 │ LocalAnomalyMap[… │
│ (GlobalAveragePool… │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,496,434 (17.15 MB)

 Trainable params: 141,634 (553.26 KB)

 Non-trainable params: 4,354,800 (16.61 MB)

In [38]:
y_train=data_train['tag'].values

In [39]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [early_stopping]
print("here")
import time, datetime
start = datetime.datetime.now()
print(start)
history=model.fit(train_cnn_data, y_train, epochs=10,callbacks=callbacks_list,batch_size=32,validation_split=0.1 )
end = datetime.datetime.now()
diff1= (end - start)
print('time taken by text_6',diff1)

here
2024-12-04 11:06:11.973078
Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/optimizers/base_optimizer.py:664: UserWarning: Gradients do not exist for variables ['kernel', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


195/195 ━━━━━━━━━━━━━━━━━━━━ 24s 106ms/step - loss: 0.3051 - val_loss: 0.2201
Epoch 2/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 19s 98ms/step - loss: 0.1860 - val_loss: 0.2135
Epoch 3/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 20s 104ms/step - loss: 0.1238 - val_loss: 0.2642
Epoch 4/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 19s 100ms/step - loss: 0.0760 - val_loss: 0.2939
Epoch 5/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 19s 98ms/step - loss: 0.0459 - val_loss: 0.3355
Epoch 5: early stopping
time taken by text_6 0:01:42.848323


In [40]:
pred=model.predict(test_cnn_data)
y_test=pred
y_test=y_test
output_class_pred=[]
for i in range(len(y_test)):
    if(y_test[i][0]<0.5):
        output_class_pred.append(0)
    else:
        output_class_pred.append(1)

original_ans=data_test['tag']
original_ans=original_ans.tolist()

57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step


In [41]:
pred = model.predict(test_cnn_data)
y_test = pred  # No need to call .tolist()

# If you need to process y_test further, you can directly use it
output_class_pred = []
for i in range(len(y_test)):
    # Continue processing y_test as needed
    output_class_pred.append(y_test[i])  # or any other logic

57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step


In [42]:
print(pred[0])

[[0.03528712]
 [0.00381404]
 [0.33960786]
 ...
 [0.00608356]
 [0.6287176 ]
 [0.02699127]]


In [43]:
original_ans=data_test['tag']
original_ans=original_ans.tolist()

In [44]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [45]:
# Assuming your test data is `test_cnn_data`
# Predict with the trained model
predictions = model.predict(test_cnn_data)

# Extract predictions from the output
normal_scores_pred = predictions[0]  # Shape (num_samples, 1)
anomalous_scores_pred = predictions[1]  # Shape (num_samples, 1)
global_anomaly_map_pred = predictions[2]  # Shape (num_samples, 1)

# Optionally: You can use a threshold to classify anomalies
# E.g., classify as anomalous if the score is above 0.5 (for binary classification)
normal_predictions = (normal_scores_pred > 0.5).astype(int)
anomalous_predictions = (anomalous_scores_pred > 0.5).astype(int)

# Global anomaly score threshold
global_anomaly_predictions = (global_anomaly_map_pred > 0.5).astype(int)

57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step


In [46]:
# Predict on test data
normal_pred, anomalous_pred, global_anomaly_map_pred = model.predict(test_cnn_data)

# Compute loss for each output manually
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError

normal_loss = BinaryCrossentropy()(y_test[0], normal_pred)
anomalous_loss = BinaryCrossentropy()(y_test[1], anomalous_pred)
global_anomaly_map_loss = MeanSquaredError()(y_test[2], global_anomaly_map_pred)

# Print the individual losses
print("Normal Loss: ", normal_loss.numpy())
print("Anomalous Loss: ", anomalous_loss.numpy())
print("Global Anomaly Map Loss: ", global_anomaly_map_loss.numpy())

57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step
Normal Loss:  0.18990242
Anomalous Loss:  0.18990242
Global Anomaly Map Loss:  0.0


In [47]:
# Flatten the predictions if necessary
normal_predictions = normal_predictions.ravel()
anomalous_predictions = anomalous_predictions.ravel()

# Apply a threshold to convert probabilities to binary labels
normal_predictions_binary = (normal_predictions > 0.5).astype(int)
anomalous_predictions_binary = (anomalous_predictions > 0.5).astype(int)

In [48]:
normal_predictions_binary , anomalous_predictions_binary

(array([0, 0, 0, ..., 0, 1, 0]), array([0, 0, 0, ..., 0, 1, 0]))

In [49]:
# Assuming y_test_normal is continuous (e.g., probabilities or scores)
y_test_normal_binary = (y_test_normal > 0.5).astype(int)

from sklearn.metrics import precision_score, recall_score, f1_score

# Calculate Precision, Recall, and F1-Score for Normal Class
precision_normal = precision_score(y_test_normal_binary, normal_predictions_binary)
recall_normal = recall_score(y_test_normal_binary, normal_predictions_binary)
f1_normal = f1_score(y_test_normal_binary, normal_predictions_binary)

# Print the results for normal class
print(f"Normal Precision: {precision_normal}")
print(f"Normal Recall: {recall_normal}")
print(f"Normal F1-Score: {f1_normal}")

Normal Precision: 0.7750257997936016
Normal Recall: 0.8261826182618262
Normal F1-Score: 0.7997870074547391
